In [1]:
import json

In [2]:
with open("training.json") as training:
    training = json.loads(training.read())
    
with open("testing.json") as testing:
    testing = json.loads(testing.read())

In [99]:
training_dict = {}
testing_dict = {}

for user in training.keys():
    for [item, rating] in training[user]:
        if user in training_dict:
            training_dict[user][item] = rating
        else:
            training_dict[user] = {item: rating}
            
for user in testing.keys():
    for [item, rating] in testing[user]:
        if user in testing_dict:
            testing_dict[user][item] = rating
        else:
            testing_dict[user] = {item: rating}

# print(training_dict['A12R54MKO17TW0'])

In [100]:
def sub_row_mean(ratings_dict):
    sub_row_mean_ratings_dict = {}
    for user in ratings_dict:
        list_item = list(ratings_dict[user].values())
        avg = sum(list_item) / len(list_item)
        for item in ratings_dict[user].keys():
            if user in sub_row_mean_ratings_dict:
                sub_row_mean_ratings_dict[user][item] = ratings_dict[user][item] - avg
            else:
                sub_row_mean_ratings_dict[user] = {item: ratings_dict[user][item] - avg}
    
    return sub_row_mean_ratings_dict
        
# training_dict2 = sub_row_mean(training_dict)
# print(training_dict2['A12R54MKO17TW0'])
# print(training_dict['A12R54MKO17TW0'])

In [101]:
import numpy as np
from numpy.linalg import norm
import math

def cosine_sim(ratings_dict, user):
    cosine_sim_val = {}

    for other_user in ratings_dict.keys():
        dot_prod = sum(ratings_dict[user][rating]*ratings_dict[other_user].get(rating, 0) for rating in ratings_dict[user])
        user_norm = norm(np.array([rating for rating in ratings_dict[user].values()]))
        other_user_norm = norm(np.array([rating for rating in ratings_dict[other_user].values()]))
        norm_mul = user_norm * other_user_norm
        cos_sim = dot_prod / norm_mul
        if math.isnan(cos_sim):
            cos_sim = 0
        cosine_sim_val[other_user] = cos_sim
    
    return cosine_sim_val

In [102]:
# k = 2
# item = "B0048P1XXQ"
# user = "A12R54MKO17TW0"
# cos_sims = cosine_sim(training_dict2, 'A12R54MKO17TW0')

def predict_item_rating_avg(ratings_dict, sub_row_mean_ratings_dict, cos_similarities_dict, k, item, user):
    # df.dropna(subset = [item])
    # df = df[df.reviewerID != user]
    # new_df =  df.nlargest(k, "cosine_sim")
    # return new_df[item].mean(axis = 0)
    cos_similarities_dict.pop(user)
    other_users = list(cos_similarities_dict.keys())
    for other_user in other_users:
        if item not in ratings_dict[other_user]:
            cos_similarities_dict.pop(other_user)
    
    list_cos_sim = sorted(cos_similarities_dict.items(), key=lambda x: x[1], reverse=True)[:k]
    print(list_cos_sim)
    avg_rating = 0
    for pair in list_cos_sim:
        avg_rating += ratings_dict[pair[0]][item]
    avg_rating = avg_rating / k
    
    return avg_rating

# training_dict["AAE2DUEMTR30I"][item] = 4.0
# print(predict_item_rating_avg(training_dict, training_dict2, cos_sims, k, item, user))
# print(training_dict["AAE2DUEMTR30I"])
# print(training_dict["AQ6J4B5WLGRJ5"])
    
    

In [103]:
# k = 2
# item = "B0048P1XXQ"
# user = "A12R54MKO17TW0"
# cos_sims = cosine_sim(training_dict2, 'A12R54MKO17TW0')

def predict_item_rating_sim(ratings_dict, sub_row_mean_ratings_dict, cos_similarities_dict, k, item, user):
    # df.dropna(subset = [item])
    # df = df[df.reviewerID != user]
    # new_df =  df.nlargest(k, "cosine_sim")
    # return (new_df["cosine_sim"].dot(new_df[item]))/(new_df["cosine_sim"].sum())
    if user in cos_similarities_dict:
        cos_similarities_dict.pop(user)
    other_users = list(cos_similarities_dict.keys())
    for other_user in other_users:
        if item not in ratings_dict[other_user]:
            cos_similarities_dict.pop(other_user)
    
    list_cos_sim = sorted(cos_similarities_dict.items(), key=lambda x: x[1], reverse=True)[:k]
    rating = 0
    similarities = 0
    for pair in list_cos_sim:
        rating += ratings_dict[pair[0]][item] * cos_similarities_dict[pair[0]]
        similarities += cos_similarities_dict[pair[0]]
    rating = rating / similarities
    
    return rating

# training_dict["AAE2DUEMTR30I"][item] = 5.0
# print(predict_item_rating_sim(training_dict, training_dict2, cos_sims, k, item, user))
# print(training_dict["AAE2DUEMTR30I"])
# print(training_dict["AQ6J4B5WLGRJ5"])
    
    

In [104]:
def predict(ratings_dict, user, item):
    sub_mean_dict = sub_row_mean(ratings_dict)
    cos_sim_dict = cosine_sim(sub_mean_dict, user)
    print(predict_item_rating_avg(training_dict, sub_mean_dict, cos_sim_dict, k, item, user))
    print(predict_item_rating_sim(training_dict, sub_mean_dict, cos_sim_dict, k, item, user))

In [111]:
# import timeit

### TEST ###

k = 2
item = "B0048P1XXQ"
user = "A12R54MKO17TW0"

# start = timeit.timeit()
predict(training_dict, user, item)
# end = timeit.timeit()

# print(end - start) => 0.0011411383748054504

<ipython-input-101-0a97a164586d>:13: RuntimeWarning: invalid value encountered in double_scalars
  cos_sim = dot_prod / norm_mul


[('AAE2DUEMTR30I', 0.037507502250750294), ('AQ6J4B5WLGRJ5', 0.023390037142659994)]
5.0
4.999999999999999
